In [12]:
############# READ DATA
import pandas as pd, numpy as np
TRAIN_PATH = 'input/saftey_efficay_myopiaTrain.csv'
TEST_PATH = 'input/saftey_efficay_myopiaTest.csv'

train = pd.read_csv(TRAIN_PATH, low_memory=False)
train = train.dropna(axis=0, how='all') # remove missing rows
print("train.shape =", train.shape)

y = train['Class']
classes_count = y.value_counts()
print('clsses counts:\nclass = 0: {} {}\nclass = 1: {} {}'.format(classes_count[0], 100*classes_count[0]/train.shape[0], classes_count[1], 100*classes_count[1]/train.shape[0]))


train.shape = (30451, 52)
clsses counts:
class = 0: 29949 98.3514498702834
class = 1: 502 1.6485501297165939


In [2]:
#### preprocess
X = train.drop(columns='Class')
dummies = pd.get_dummies(X)
X=(dummies-dummies.min())/(dummies.max()-dummies.min())
X = X.fillna(value=0)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)





In [18]:
from sklearn.ensemble import RandomForestClassifier
foo = RandomForestClassifier(n_estimators=100, criterion='entropy')
foo.fit(X_train, y_train)
pred = foo.predict(X_test)



In [3]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [9]:
y_pred = model.predict(X_test)
ans = y_pred > 0
sum(ans)

C:\Users\Eli\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0